In [1]:
%matplotlib inline

In [2]:
import datetime, time
import lxml

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
seqs = pd.read_csv("../data/sequential_id.csv")

In [5]:
seqs['name'] = seqs.name.str.upper()
seqs['name'] = seqs.name.replace('AURÉLIO NOMURA', 'AURELIO NOMURA')
seqs['name'] = seqs.name.replace('CAIO MIRANDA CARNEIRO', 'CAIO MIRANDA')
seqs['name'] = seqs.name.replace('ANTONIO DONATO', 'DONATO')
seqs['name'] = seqs.name.replace('EDUARDO SUPLICY', 'EDUARDO MATARAZZO SUPLICY') 
seqs['name'] = seqs.name.replace('MARIO COVAS NETO', 'MÁRIO COVAS NETO')
seqs['name'] = seqs.name.replace('REGINALDO TRIPOLI', 'REGINALDO TRÍPOLI')
seqs['name'] = seqs.name.replace('ZÉ TURIN', 'ZÉ TURIM')

In [6]:
xmls = [] 
xmls.append('http://www2.camara.sp.gov.br/sisgv/Arquivos/201707.XML')

In [7]:
lista_gastos=[]
for link in xmls:
    r = requests.get(link)
    xml = BeautifulSoup(r.content, 'xml')
    gastos = xml.find_all('TabelaPortalITEMREEMBOLSO')
    for s in gastos:
        chave = s.Chave.text
        centro_custos = s.CENTROCUSTOSID.text
        departamento = s.DEPARTAMENTO.text
        tipodep = s.TIPODEDEPARTAMENTO
        vereador = s.VEREADOR.text
        ano = s.ANO.text
        mes = s.MES.text
        despesa = s.DESPESA.text
        cnpj = s.CNPJ.text
        fornecedor = s.FORNECEDOR.text
        valor = s.VALOR.text
        data = datetime.date(int(ano), int(mes), 1)

        dict_gastos = {'DATA': data, 'Chave':chave, 'CENTROCUSTOSID': centro_custos, 'DEPARTAMENTO': departamento, 
                'TIPO_DEPARTAMENTO':tipodep, 'VEREADOR': vereador, 'DESPESA':despesa, 'CNPJ': cnpj, 
                       'FORNECEDOR': fornecedor, 'VALOR': valor}
        lista_gastos.append(dict_gastos)

In [8]:
df = pd.DataFrame().from_records(lista_gastos)

In [9]:
df = df[['DATA', 'VEREADOR', 'DEPARTAMENTO', 'DESPESA', 'VALOR', 'FORNECEDOR', 'CNPJ']]
df[['DATA']] = df[['DATA']].apply(pd.to_datetime)
df[['VALOR']] = df[['VALOR']].apply(pd.to_numeric)

In [10]:
df = pd.merge(df, seqs, how='left', left_on='VEREADOR', right_on='name')

In [11]:
df = df.drop(['VEREADOR', 'name'], axis=1)

In [12]:
df = df[df.sequential_id.notnull()]

In [13]:
df = df.rename(columns={
    'DATA': 'date', 
    'VEREADOR': 'candidate',
    'DEPARTAMENTO': 'department', 
    'DESPESA': 'expense',
    'VALOR': 'value', 
    'FORNECEDOR': 'provider', 
    'CNPJ': 'cnpj',
})

In [14]:
df.tail()

,date,department,expense,value,provider,cnpj,sequential_id
1955,2017-05-01,GABINETE DE VEREADOR,LIVROS,562.90,LIVRARIA CULTURA s/a,62.410.352/0006-87,2.500000e+11
1956,2017-05-01,GABINETE DE VEREADOR,MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE C...,131.70,KALUNGA COM.E IND.GRÁFICA LTDA,43.283.811/0085-68,2.500000e+11
1957,2017-05-01,GABINETE DE VEREADOR,MATERIAL DE ESCRITORIO E OUTROS MATERIAIS DE C...,8.90,SARAIVA E SICILIANO S/A,61.365.284/0107-62,2.500000e+11
1958,2017-05-01,GABINETE DE VEREADOR,TELEFONE FIXO,168.82,TELEFONICA BRASIL S/A,02.558.157/0001-62,2.500000e+11
1959,2017-05-01,GABINETE DE VEREADOR,TELEFONE MOVEL,177.24,OI MÓVEL S.A.,05.423.963/0026-70,2.500000e+11


In [15]:
df.to_csv(f"../data/expenses.csv", index=False)